# Project Cloud Data Warehouse

# Import Libraries

In [ ]:
import pandas as pd
import boto3
import json
import configparser

# Load DWH Params from a file

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

LOG_DATA               = config.get('S3','LOG_DATA')
LOG_JSONPATH           = config.get('S3','LOG_JSONPATH')
SONG_DATA              = config.get('S3','SONG_DATA')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

# Create clients for IAM, EC2, S3 and Redshift
**Note**: We are creating these resources in the the **us-west-2** region. Choose the same region in the your AWS web console to the see these resources.

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# Check out the log-data sources on S3

In [ ]:
DWHBucket =  s3.Bucket("udacity-dend")
log_data_files = [filename.key for filename in DWHBucket.objects.filter(Prefix='log-data')]
log_data_files[:10]

# Check out the song_data sources on S3

In [ ]:
DWHBucket =  s3.Bucket("udacity-dend")
song_data_files = [filename.key for filename in DWHBucket.objects.filter(Prefix='song_data')]
song_data_files[:10]

# STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

# STEP 2:  Redshift Cluster

- Create a [RedShift Cluster](https://console.aws.amazon.com/redshiftv2/home)
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## 2.1 Describe the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available". Make ure you are checking your Amazon Redshift cluster in the **us-west-2** region. </font>


- Get endpoint & IAM role from cluster and update dwh.cfg accordingly

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
config.set('CLUSTER','host', DWH_ENDPOINT)
config.set('IAM_ROLE','arn', DWH_ROLE_ARN)
with open('dwh.cfg', 'w') as configfile:
    config.write(configfile)

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

# STEP 4: Make sure you can connect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

# STEP 5: Create Tables

In [ ]:
%%sql 
DROP TABLE IF EXISTS staging_events;
DROP TABLE IF EXISTS staging_songs;
DROP TABLE IF EXISTS songplays;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS song;
DROP TABLE IF EXISTS artist;
DROP TABLE IF EXISTS time;

CREATE TABLE staging_events(
    artist VARCHAR,
    auth VARCHAR,
    firstName VARCHAR,
    gender CHAR(1),
    itemInSession INTEGER,
    lastName VARCHAR,
    length DECIMAL,
    level VARCHAR,
    location VARCHAR,
    method VARCHAR,
    page VARCHAR,
    registration BIGINT,
    sessionID INTEGER,
    song VARCHAR,
    status INTEGER,
    ts TIMESTAMP,
    userAgent VARCHAR,
    userId INTEGER
);

CREATE TABLE staging_songs(
    num_songs INTEGER,
    artist_id VARCHAR,
    artist_latitude DECIMAL,
    artist_longitude DECIMAL,
    artist_location VARCHAR,
    artist_name VARCHAR,
    song_id VARCHAR,
    title VARCHAR,
    duration DECIMAL,
    year INTEGER 
);

CREATE TABLE songplays(
    songplay_id INTEGER IDENTITY(0,1) NULL sortkey,
    start_time TIMESTAMP,
    user_id INTEGER,
    level VARCHAR,
    song_id VARCHAR,
    artist_id VARCHAR,
    session_id INTEGER,
    location VARCHAR,
    user_agent VARCHAR)
diststyle all;

CREATE TABLE users(
    user_id INTEGER  PRIMARY KEY distkey,
    first_name VARCHAR,
    last_name VARCHAR,
    gender CHAR(1),
    level VARCHAR 
);

CREATE TABLE songs(
    song_id VARCHAR  PRIMARY KEY,
    title VARCHAR,
    artist_id VARCHAR distkey,
    year INTEGER,
    duration DECIMAL 
);

CREATE TABLE artists(
    artist_id VARCHAR  PRIMARY KEY distkey,
    name VARCHAR,
    location VARCHAR,
    latitude DECIMAL,
    longitude DECIMAL 
);
CREATE TABLE time(
    start_time TIMESTAMP  PRIMARY KEY distkey,
    hour INTEGER,
    day INTEGER,
    week INTEGER,
    month INTEGER,
    year INTEGER,
    weekday INTEGER 
);

# STEP 6: Load Tables

## Load data from S3 into staging_events

In [ ]:
%%time
qry = """
    COPY staging_events 
    FROM {}
    CREDENTIALS 'aws_iam_role={}'
    COMPUPDATE OFF
    REGION 'us-west-2'
    TIMEFORMAT AS 'epochmillisecs'
    TRUNCATECOLUMNS BLANKSASNULL EMPTYASNULL
    JSON {};
""".format(LOG_DATA, DWH_ROLE_ARN, LOG_JSONPATH)

%sql $qry

In [ ]:
%sql select count(*) from staging_events

## Load data from S3 into staging_songs

In [ ]:
%%time
qry = """
    COPY staging_songs FROM {}
    CREDENTIALS 'aws_iam_role={}'
    COMPUPDATE OFF region 'us-west-2'
    FORMAT AS JSON 'auto' 
    TRUNCATECOLUMNS BLANKSASNULL EMPTYASNULL;
""".format(SONG_DATA, DWH_ROLE_ARN)

%sql $qry

In [ ]:
%sql select count(*) from staging_songs

## Load data from staging_events & staging_songs into songplays

In [ ]:
%%sql
INSERT INTO songplays(start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT  TO_DATE(se.ts,'dd.mm.yyyy/hh:mi:ss') AS start_time,
        se.userId                            AS user_id,
        se.level                             AS level,
        ss.song_id                           AS song_id,
        ss.artist_id                         AS artist_id,
        se.sessionID                         AS session_id,
        se.location                          AS location,
        se.userAgent                         AS user_agent
FROM staging_events se
JOIN staging_songs ss ON (se.song = ss.title) AND (se.artist = ss.artist_name)
WHERE se.ts IS NOT NULL;

## Load data from staging_events into users

In [ ]:
%%sql
INSERT INTO users(user_id, first_name, last_name, gender, level)
SELECT  se.userId    AS user_id,
        se.firstName AS first_name,
        se.lastName  AS last_name,
        se.gender    AS gender,
        se.level     AS level
FROM staging_events se
WHERE se.userId IS NOT NULL;

## Load data from staging_songs into songs

In [ ]:
%%sql
INSERT INTO songs(song_id, title, artist_id, year, duration)
SELECT  ss.song_id      AS song_id,
        ss.title        AS title,
        ss.artist_id    AS artist_id,
        ss.year         AS year,
        ss.duration     AS duration
FROM staging_songs ss
WHERE ss.song_id IS NOT NULL;

## Load data from staging_songs into artist

In [ ]:
%%sql
INSERT INTO artists(artist_id, name, location, latitude, longitude)
SELECT  ss.artist_id        AS artist_id,
        ss.artist_name      AS name,
        ss.artist_location  AS location,
        ss.artist_latitude  AS latitude,
        ss.artist_longitude AS longitude
FROM staging_songs ss
WHERE ss.artist_id IS NOT NULL;

## Load data from staging_events into time

In [ ]:
%%sql
INSERT INTO time(start_time, hour, day, month, week, year, weekday)
SELECT  TO_DATE(se.ts,'dd.mm.yyyy/hh:mi:ss') AS start_time,
        DATEPART(HOUR,start_time)            AS hour,
        DATEPART(DAY,start_time)             AS day,
        DATEPART(MONTH,start_time)           AS month,
        DATEPART(WEEK,start_time)            AS week,
        DATEPART(YEAR,start_time)            AS year,
        DATEPART(WEEKDAY,start_time)         AS weekday
FROM staging_events se
WHERE se.ts IS NOT NULL;

# STEP 7: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

## Verify that resources have been deleted properly

- run this block several times until the cluster really deleted

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!